In [23]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import preprocessing #Convert categorical data to neumeric values
from sklearn.neural_network import MLPClassifier #Model being used

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/widsdatathon2023/train_data.csv
/kaggle/input/widsdatathon2023/test_data.csv
/kaggle/input/widsdatathon2023/sample_solution.csv


In [24]:
keep_columns=['lat','month','contest-tmp2m-14d__tmp2m',
              'climateregions__climateregion','lon']
X_columns=[['lat','month',
              'climateregions__climateregion','lon']]

In [25]:
def seperate_monthly(df):
    df[['month','day','year']]=df.startdate.str.split('/',expand=True)
    df=df.drop('day', axis=1)
    df=df.drop('startdate',axis=1)
    return df

In [26]:
def eliminate_columns(df):
    for col in df.columns:
        if col in keep_columns:
            print(col)
        else: 
            df=df.drop(col, axis=1)
    return df

In [27]:
#need to come up with a different way to do this, because it is not working
def seperate_y(df):
    y_df=df['contest-tmp2m-14d__tmp2m']
    X_df=df.drop('contest-tmp2m-14d__tmp2m',axis=1)
    return y_df,X_df

In [28]:
def normanlize_data(df,train_df):
    for column in train_df.columns:
        df[column] = train_df[column]  /  train_df[column].abs().max()

In [29]:
train_df=pd.read_csv('/kaggle/input/widsdatathon2023/train_data.csv')
train_df=train_df.dropna(axis=0)
test_df=pd.read_csv('/kaggle/input/widsdatathon2023/test_data.csv')
test_df=test_df.dropna(axis=0)
train_df

,index,lat,lon,startdate,contest-pevpr-sfc-gauss-14d__pevpr,nmme0-tmp2m-34w__cancm30,nmme0-tmp2m-34w__cancm40,nmme0-tmp2m-34w__ccsm30,nmme0-tmp2m-34w__ccsm40,nmme0-tmp2m-34w__cfsv20,...,wind-vwnd-925-2010-11,wind-vwnd-925-2010-12,wind-vwnd-925-2010-13,wind-vwnd-925-2010-14,wind-vwnd-925-2010-15,wind-vwnd-925-2010-16,wind-vwnd-925-2010-17,wind-vwnd-925-2010-18,wind-vwnd-925-2010-19,wind-vwnd-925-2010-20
0,0,0.0,0.833333,9/1/14,237.00,29.02,31.64,29.57,30.73,29.71,...,-27.68,-37.21,8.32,9.56,-2.03,48.13,28.09,-13.50,11.90,4.58
1,1,0.0,0.833333,9/2/14,228.90,29.02,31.64,29.57,30.73,29.71,...,-21.13,-36.57,8.77,21.17,4.44,48.60,27.41,-23.77,15.44,3.42
2,2,0.0,0.833333,9/3/14,220.69,29.02,31.64,29.57,30.73,29.71,...,-10.72,-34.16,6.99,32.16,5.01,48.53,19.21,-33.16,15.11,4.82
3,3,0.0,0.833333,9/4/14,225.28,29.02,31.64,29.57,30.73,29.71,...,0.33,-31.04,6.17,39.66,-1.41,50.59,8.29,-37.22,18.24,9.74
4,4,0.0,0.833333,9/5/14,237.24,29.02,31.64,29.57,30.73,29.71,...,9.83,-31.80,7.47,38.62,-5.21,54.73,-2.58,-42.30,21.91,10.95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375729,375729,1.0,0.866667,8/27/16,312.05,23.13,27.20,20.25,24.43,18.35,...,-15.64,-75.68,-3.09,6.93,-16.69,16.98,-13.85,50.25,-31.33,0.77
375730,375730,1.0,0.866667,8/28/16,305.82,23.13,27.20,20.25,24.43,18.35,...,-7.59,-76.42,-13.55,13.36,-15.96,20.45,-16.36,51.65,-30.73,10.10
375731,375731,1.0,0.866667,8/29/16,311.62,23.13,27.20,20.25,24.43,18.35,...,-6.25,-70.65,-23.93,22.62,-16.71,20.28,-15.48,48.58,-18.74,9.28
375732,375732,1.0,0.866667,8/30/16,304.54,23.13,27.20,20.25,24.43,18.35,...,-7.16,-57.67,-33.55,32.06,-16.07,16.60,-20.61,39.23,-16.26,-0.22


In [30]:
train_df=train_df[1000:]
test_df=test_df[:375600]
train_df

,index,lat,lon,startdate,contest-pevpr-sfc-gauss-14d__pevpr,nmme0-tmp2m-34w__cancm30,nmme0-tmp2m-34w__cancm40,nmme0-tmp2m-34w__ccsm30,nmme0-tmp2m-34w__ccsm40,nmme0-tmp2m-34w__cfsv20,...,wind-vwnd-925-2010-11,wind-vwnd-925-2010-12,wind-vwnd-925-2010-13,wind-vwnd-925-2010-14,wind-vwnd-925-2010-15,wind-vwnd-925-2010-16,wind-vwnd-925-2010-17,wind-vwnd-925-2010-18,wind-vwnd-925-2010-19,wind-vwnd-925-2010-20
1045,1045,0.0,0.866667,7/12/15,536.48,27.18,31.74,26.38,28.36,26.69,...,31.42,-44.61,-45.24,50.18,19.49,-14.69,-11.09,20.40,18.55,4.07
1046,1046,0.0,0.866667,7/13/15,542.93,27.18,31.74,26.38,28.36,26.69,...,17.07,-43.80,-38.24,38.52,20.25,-17.89,-14.78,22.72,13.80,-5.87
1047,1047,0.0,0.866667,7/14/15,548.13,27.18,31.74,26.38,28.36,26.69,...,4.34,-48.78,-34.02,28.81,25.96,-19.85,-15.82,28.41,5.04,-9.28
1048,1048,0.0,0.866667,7/15/15,549.71,27.18,31.74,26.38,28.36,26.69,...,-4.55,-53.35,-32.99,20.65,32.62,-24.13,-13.79,32.38,-7.66,-12.48
1049,1049,0.0,0.866667,7/16/15,552.76,27.18,31.74,26.38,28.36,26.69,...,-6.16,-59.54,-32.29,14.75,36.98,-27.01,-9.04,33.19,-23.54,-9.55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375729,375729,1.0,0.866667,8/27/16,312.05,23.13,27.20,20.25,24.43,18.35,...,-15.64,-75.68,-3.09,6.93,-16.69,16.98,-13.85,50.25,-31.33,0.77
375730,375730,1.0,0.866667,8/28/16,305.82,23.13,27.20,20.25,24.43,18.35,...,-7.59,-76.42,-13.55,13.36,-15.96,20.45,-16.36,51.65,-30.73,10.10
375731,375731,1.0,0.866667,8/29/16,311.62,23.13,27.20,20.25,24.43,18.35,...,-6.25,-70.65,-23.93,22.62,-16.71,20.28,-15.48,48.58,-18.74,9.28
375732,375732,1.0,0.866667,8/30/16,304.54,23.13,27.20,20.25,24.43,18.35,...,-7.16,-57.67,-33.55,32.06,-16.07,16.60,-20.61,39.23,-16.26,-0.22


In [31]:
#Shuffeling the data
train_df = train_df.sample(frac = 1)

In [32]:
le = preprocessing.LabelEncoder()

# Converting string labels into numbers.
train_df['climateregions__climateregion']=le.fit_transform(train_df['climateregions__climateregion'])
test_df['climateregions__climateregion']=le.fit_transform(test_df['climateregions__climateregion'])

In [33]:
print('training set')
train_df=seperate_monthly(train_df)
train_df=eliminate_columns(train_df)
print('test set')
test_df=seperate_monthly(test_df)
test_df=eliminate_columns(test_df)

training set
lat
lon
contest-tmp2m-14d__tmp2m
climateregions__climateregion
month
test set
lat
lon
climateregions__climateregion
month


In [34]:
train_df

,lat,lon,contest-tmp2m-14d__tmp2m,climateregions__climateregion,month
319341,0.909091,0.066667,11.522637,7,5
65541,0.318182,0.666667,-0.838191,1,12
138335,0.500000,0.366667,0.778214,5,2
263511,0.772727,0.300000,16.107005,12,8
353583,0.954545,0.700000,-2.438851,9,1
...,...,...,...,...,...
18271,0.181818,0.733333,26.965667,0,8
283938,0.818182,0.300000,17.584608,9,7
221499,0.681818,0.266667,16.756770,1,9
1785,0.045455,0.833333,31.967969,4,7


In [35]:
train_df=train_df.astype('float')
test_df=test_df.astype('float')

In [36]:
y_train_df=train_df['contest-tmp2m-14d__tmp2m']
X_train_df=train_df.drop('contest-tmp2m-14d__tmp2m', axis=1)
type(y_train_df)
y_train_df=y_train_df.to_frame()
type(y_train_df)

pandas.core.frame.DataFrame

In [37]:
X_train_df=normanlize_data(X_train_df,train_df)
test_df=normanlize_data(test_df,train_df)

In [38]:
y_train_df=1000*y_train_df

In [39]:
y_train_df

,contest-tmp2m-14d__tmp2m
319341,11522.636550
65541,-838.190824
138335,778.213535
263511,16107.004910
353583,-2438.850990
...,...
18271,26965.666910
283938,17584.607940
221499,16756.769520
1785,31967.969080


In [40]:
#Idk why but apparentley I need an integer
y_train_df=y_train_df.astype('int')
y_train_df

,contest-tmp2m-14d__tmp2m
319341,11522
65541,-838
138335,778
263511,16107
353583,-2438
...,...
18271,26965
283938,17584
221499,16756
1785,31967


In [41]:
clf = MLPClassifier(hidden_layer_sizes=(2,10),
                    random_state=100,
                    verbose=True,
                    learning_rate_init=0.01)

# Fit data onto the model
clf.fit(X_train_df,y_train_df)

ValueError: Expected 2D array, got scalar array instead:
array=nan.
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [ ]:
ypred=clf.predict(test_df)

In [ ]:
#Trying to create a final 
final=pd.DataFrame()
final['contest-tmp2m-14d__tmp2m']=ypred
final['index']=0
i=375734 #Legnth of train_data.csv
while i<407088:
    final['index'][i-375734]=i
    i+=1
final

In [ ]:
final.to_csv('final.csv',index=False)

In [ ]:
final